## Train Test Split

In [4]:
# %run "01.Recommendation_Loading.ipynb"

################  Recommendations (all data, positive recommendations only) ################
Users: 12,636,209
Games: 37,419
Number of users 10,000
First five users id [11203022 12827342 11719330  9553563 11051045]
################  Recommendations (Sample) ################
Minimum number of recommendations to included in the sample: 20
Number of randomly selected users: 10000
Users: 9,688
Games: 20,615
################ User-Game Matrix ################
Number of Rows: 9,689
Numbeer of Columns: 20,616
Number of stored values: 365,753


### Test Set

In [5]:
# def get_test_instances(test):
def get_test_instances(df):
    """
    This functions generates test instances using the Leave One Out Approach taking the maximum recommendation date.
    There could be more than 1 recommendations in the same day. In that case, take the max game id
    """
    max_df = df.groupby("user_id_categorical").agg({"date": np.nanmax}).reset_index()
    rec_df = pd.merge(df, max_df, left_on=["user_id_categorical","date"], right_on=["user_id_categorical","date"])
    rec_df = df.groupby("user_id_categorical").agg({"app_id_categorical": np.nanmax}).reset_index()
    # rec_df["user_id_categorical_new"] = rec_df["user_id_categorical"].map(swapped_users_dict)
    # rec_df["app_id_categorical_new"] = rec_df["app_id_categorical"].map(swapped_games_dict)
    # x_test = rec_df[["user_id_categorical_new", "app_id_categorical_new"]].values
    x_test = rec_df[["user_id_categorical", "app_id_categorical"]].values
    y_test = np.full((x_test.shape[0],1), 1)
    return x_test, y_test

print("################  Get Test Data ################")
x_test, y_test = get_test_instances(recommendations)
print("x_test.shape:",  x_test.shape)
print("y_test.shape:",  y_test.shape)


################  Get Test Data ################
x_test.shape: (9688, 2)
y_test.shape: (9688, 1)


In [6]:
x_test_dict = dict(x_test)

In [ ]:
# x_test[-10:]

### Get Training Istances

Including negative samples

In [7]:
NUM_NEGATIVES = 2

In [8]:
num_users, num_items = user_game_matrix.shape
print("Number of Users:", '{0:,.0f}'.format(num_users))
print("Number of Games:", '{0:,.0f}'.format(num_items))
print("Number of Negatives Sample per User-Game:", '{0:,.0f}'.format(NUM_NEGATIVES))


Number of Users: 9,689
Number of Games: 20,616
Number of Negatives Sample per User-Game: 2


In [9]:
def get_train_instances(train, num_negatives, gen_negative=True):
    """
    This function generates training instances. For each user in the training data,
    I should have users who recommended games and users who do not recommend games
    """
    user_input, item_input, labels = [],[],[]
    for (u, i) in user_game_matrix.keys():
        # positive instance
        if (u,i) not in x_test_dict.keys(): # The test instance must not be included
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
            if gen_negative==True:
                # negative instances
                item_selected = [] # to avoid duplicates
                for t in range(num_negatives):
                    j = np.random.randint(num_items)
                    while ((u, j) in user_game_matrix.keys()) \
                        or ((u,j) in item_selected) \
                        or ((u,j) in x_test_dict.keys()):
                        j = np.random.randint(num_items)
                    user_input.append(u)
                    item_input.append(j)
                    labels.append(0)
                    item_selected.append((u,j))
        
    p = np.vstack([user_input, item_input, labels]).T
    p = np.unique(p, axis=0)
    x_train = p[:,:2]
    y_train = p[:, 2].reshape(-1, 1)
    return x_train, y_train

print("################  Get Training Data ################")
x_train, y_train = get_train_instances(user_game_matrix, NUM_NEGATIVES)
print("x_train.shape:",  x_train.shape)
print("y_train.shape:",  x_train.shape)


################  Get Training Data ################
x_train.shape: (1094928, 2)
y_train.shape: (1094928, 2)


In [ ]:
# x_train[:100]

In [ ]:
# y_train[:100]

In [11]:
# user_game_matrix[0,:].keys()

## Model Evaluation

In [12]:
def get_random_userid(num_users):
    """
    pick 1 random user at random
    Args:
        num_users:: int
    Returns:
        user_id_eval:: int, user_id to evaluate
    """
    user_id_eval = np.random.randint(num_users)
    return user_id_eval

In [13]:
def get_y_true(user_id, x_train, y_train):
    # Stacking x_train and y_train
    y_true_tmp = np.hstack((x_train, y_train))
    # keep only the recommended games
    y_true_tmp = y_true_tmp[y_true_tmp[:,-1] == 1]
    # filtering for the user id
    y_true = y_true_tmp[y_true_tmp[:,0] == user_id][:,1]
    return y_true


In [14]:
def get_user_games(user_id, num_games):
    # Generate a list of all games for a specific user
    #games = list(games_mapping.keys())
    games = np.arange(num_games)
    games2pred = np.reshape(games, (-1,1))
    user_game_array_eval = np.hstack(([[user_id]] * len(games), games2pred))
    return user_game_array_eval


In [15]:
def get_101_games(user_id, num_games, num_gen_games, x_train, x_test):
    seen_games = x_train[x_train[:,0]==user_id][:,1]
    games_pred = []
    for t in range(num_gen_games):
        j = np.random.randint(num_games)
        while j in seen_games:
             j = np.random.randint(num_games)
        games_pred.append(j)
    test_games = x_test[x_test[:,0]==user_id][:,1][0]
    games_pred.append(test_games)
    games2pred = np.reshape(games_pred, (-1,1))
    user_game_array_eval = np.hstack(([[user_id]] * len(games2pred), games2pred))
    return user_game_array_eval

In [ ]:
# x_train[x_train[:,0]==1][:,1]

In [ ]:
# x_test[:5]

In [ ]:
# x_test[x_test[:,0]==1][:,1]

In [ ]:
# get_101_games(2, num_items, 100, x_train, x_test)

In [ ]:
# for x in x_train[x_train[:,0]==1][:,1]:
#    print(x in get_101_games(1, num_items, 100, x_train, x_test))

In [16]:
def get_y_pred(user_games_array, K):
    preds = model.predict(user_games_array)
    user_games_preds = np.hstack((user_games_array, preds))
    sorted_indices = np.argsort(-user_games_preds[:, -1])
    y_pred = user_games_preds[sorted_indices][:K, :]
    y_pred = y_pred[:,1].astype('int32')
    return y_pred

In [17]:
def eval_k(y_true, y_pred):
    items_relevant_user = len(y_true)
    if items_relevant_user==0:
        items_relevant_user = np.NaN
    rec_items_relevant = len(set(y_true).intersection(set(y_pred)))
    recall = rec_items_relevant / items_relevant_user
    precision = rec_items_relevant / K
    return recall, precision


In [18]:
def eval_k_avg(n_users2val, K, num_users, num_games, x_train, y_train):
    """
    Evaluate Top K precision and recall
    Args:
        n_users2val:: int, number of users to evaluate
        K:: int
        num_users:: int, number of users in the test set
        games_mapping:: dict, keys: integer (idx), values: orginal app_id_categorical
        x_train
        y_train

    Returns:
        avg_precision:: float, top K average precision
        avg_recall:: float, top K average recall
    """
    precision = []
    recall = []
    for i in range(n_users2val):
        user_id_eval = get_random_userid(num_users)
        y_true = get_y_true(user_id_eval, x_train, y_train)
        user_games = get_user_games(user_id_eval, num_games)
        y_pred = get_y_pred(user_games, K)
        rcl, prc = eval_k(y_true, y_pred)
        precision.append(prc)
        recall.append(rcl)
    avg_precision = np.mean(precision)
    avg_recall = np.mean(recall)
    return avg_precision, avg_recall


In [20]:
def eval_hit_ratio(n_users2val, K, num_games, num_gen_games, x_train, x_test):
    counter = 0
    # randomly select n_users2val from the test test
    choosen_user = np.random.choice(x_test[:,0], size = n_users2val)
    # print(n_users2val)
    for user_game in x_test[choosen_user,:]:
        # print(user_game)
        # user_games = get_user_games(user_game[0], num_games)
        user_games = get_101_games(user_game[0], num_games, num_gen_games, x_train, x_test)
        y_pred = get_y_pred(user_games, K)
        if(user_game[1] in y_pred):
            counter += 1
    return counter / n_users2val

In [24]:
# def eval_hit_ratio_pop(n_users2val, pop_games, x_test):
def eval_hit_ratio_pop(pop_games, x_test):

    counter = 0
    # randomly select n_users2val from the test test
    # choosen_user = np.random.choice(x_test[:,0], size = n_users2val)
    # print(n_users2val)
    # for user_game in x_test[choosen_user,:]:
    for user_game in x_test:
        if(user_game[1] in pop_games):
            counter += 1
    # return counter / n_users2val
    return counter / x_test.shape[0]

In [ ]:
def eval_hit_ratio_sim(rec_sim, x_test):

    counter = 0
    for i, user_game in enumerate(x_test):        
        if(user_game[1] in rec_sim[i]):
            counter += 1
    # return counter / n_users2val
    return counter / x_test.shape[0]